In [2]:
!pip install apache-sedona[spark]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self

In [3]:
from pyspark.sql import SparkSession

from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer

spark = SparkSession. \
    builder. \
    appName('appName'). \
    config("spark.serializer", KryoSerializer.getName). \
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName). \
    config('spark.jars.packages',
           'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.2.0-incubating,'
           'org.datasyslab:geotools-wrapper:1.1.0-25.2'). \
    getOrCreate()

SedonaRegistrator.registerAll(spark)

True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
DATA_PATH = '/content/drive/MyDrive/sedona_osm_data'

In [6]:
sc = spark.sparkContext

In [7]:
from sedona.core.formatMapper import WktReader
import csv, sys, pprint, hashlib
from shapely import wkt
import numpy as np

### Points

In [38]:
points_rdd = WktReader.readToGeometryRDD(sc, DATA_PATH + '/all_points_100K.wkt', 1, True, False)

In [39]:
points_rdd

In [40]:
from sedona.core.enums import GridType, IndexType
from sedona.utils.adapter import Adapter

points_rdd.analyze()

True

In [41]:
len = 100000

In [42]:
points_rdd.spatialPartitioning(GridType.QUADTREE)

True

In [43]:
import time
from sedona.core.geom.envelope import Envelope
from shapely.geometry import Polygon
from sedona.core.spatialOperator import RangeQuery

# polygon = open(DATA_PATH + "/knn_polygon.tsv", "r").read()
# shape = wkt.loads(polygon)
# coords = np.dstack(shape.boundary.xy).tolist()[0][:-1]

query_envelope = Envelope(-85.01, -60.01, 34.01, 50.01)
# query_envelope = Polygon(coords)

for i in range(3):
  if i == 0:
    print("'No Index'")
    using_index = False
  elif i == 1:
    print("'R-Tree Index'")
    # using R-tree index
    using_index = True

    build_on_spatial_partitioned_rdd = False ## Set to TRUE only if run join query
    points_rdd.buildIndex(IndexType.RTREE, build_on_spatial_partitioned_rdd)
  elif i == 2:
    print("'Quad-Tree Index'")
    # using Quad-tree index
    using_index = True

    build_on_spatial_partitioned_rdd = False ## Set to TRUE only if run join query
    points_rdd.buildIndex(IndexType.QUADTREE, build_on_spatial_partitioned_rdd)


  print("'n,time(s)'")
  s = time.time()
  result = RangeQuery.SpatialRangeQuery(points_rdd, query_envelope, True, using_index)
  result.collect()
  d = time.time() - s
  print("'" + str(len) + "," + str(round(d, 3)) + "'")

'No Index'
'n,time(s)'
'100000,0.388'
'R-Tree Index'
'n,time(s)'
'100000,0.622'
'Quad-Tree Index'
'n,time(s)'
'100000,0.554'


### Polygons

In [8]:
polygon_rdd = WktReader.readToGeometryRDD(sc, DATA_PATH + '/all_source_100K.wkt', 1, True, False)

In [9]:
polygon_rdd

In [10]:
from sedona.core.enums import GridType, IndexType
from sedona.utils.adapter import Adapter

polygon_rdd.analyze()

True

In [11]:
len = 100000

In [12]:
polygon_rdd.spatialPartitioning(GridType.KDBTREE )

True

In [13]:
import time
from sedona.core.geom.envelope import Envelope
from shapely.geometry import Polygon
from sedona.core.spatialOperator import RangeQuery

# polygon = open(DATA_PATH + "/knn_polygon.tsv", "r").read()
# shape = wkt.loads(polygon)
# coords = np.dstack(shape.boundary.xy).tolist()[0][:-1]

query_envelope = Envelope(-85.01, -60.01, 34.01, 50.01)
# query_envelope = Polygon(coords)

for i in range(3):
  if i == 0:
    print("'No Index'")
    using_index = False
  elif i == 1:
    print("'R-Tree Index'")
    # using R-tree index
    using_index = True

    build_on_spatial_partitioned_rdd = False ## Set to TRUE only if run join query
    polygon_rdd.buildIndex(IndexType.RTREE, build_on_spatial_partitioned_rdd)
  elif i == 2:
    print("'Quad-Tree Index'")
    # using Quad-tree index
    using_index = True

    build_on_spatial_partitioned_rdd = False ## Set to TRUE only if run join query
    polygon_rdd.buildIndex(IndexType.QUADTREE, build_on_spatial_partitioned_rdd)


  print("'n,time(s)'")
  s = time.time()
  result = RangeQuery.SpatialRangeQuery(polygon_rdd, query_envelope, True, using_index)
  result.collect()
  d = time.time() - s
  print("'" + str(len) + "," + str(round(d, 3)) + "'")

/usr/local/lib/python3.8/dist-packages/sedona/core/geom/envelope.py:27: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self.minx = minx
/usr/local/lib/python3.8/dist-packages/sedona/core/geom/envelope.py:28: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self.maxx = maxx
/usr/local/lib/python3.8/dist-packages/sedona/core/geom/envelope.py:29: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self.miny = miny
/usr/local/lib/python3.8/dist-packages/sedona/core/geom/envelope.py:30: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self.maxy = maxy


'No Index'
'n,time(s)'
'100000,275.644'
'R-Tree Index'
'n,time(s)'
'100000,255.524'
'Quad-Tree Index'
'n,time(s)'
'100000,261.299'
